# Cut image to a few z's

In [1]:
import tifffile as tf
import numpy as np
from pathlib import Path

In [2]:
test = tf.imread('/groups/CaiLab/personal/Lex/raw/2020-08-08-takei/notebook_pyfiles/deconvoluted_images/HybCycle_0/MMStack_Pos0.ome.tif')

In [3]:
test.shape

(13, 4, 2048, 2048)

In [4]:
for i in range(65):
    zslice = []
    img = tf.imread(f'../deconvoluted_images/HybCycle_{i}/MMStack_Pos0.ome.tif')
    for z in np.arange(1,13,4):
        zslice.append(img[z])
    p = Path(f'../sliced_img/HybCycle_{i}/MMStack_Pos0.ome.tif')
    p.parent.mkdir(parents=True, exist_ok=True)
    newslice = np.array(zslice)
    tf.imwrite(str(p),newslice[:,:3,:,:])

KeyboardInterrupt: 

# Look at images

In [ ]:
tiff_src = '/groups/CaiLab/personal/Lex/raw/2020-08-08-takei/notebook_pyfiles/sliced_img/HybCycle_0/MMStack_Pos0.ome.tif'

In [ ]:
img = tf.imread(tiff_src)

In [ ]:
img.shape

In [ ]:
import plotly.express as px
def plot_slideshow(img, zmax):
    """Function to generate plots with slide panel
    Parameters:
    -----------
    img = image containing ref and corrected
    zmax= set maximum intensity"""
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img,
        width=600,
        height=600,
        binary_string=True,
        binary_compression_level=4,
        animation_frame=0,
        binary_backend='pil',
        zmax = zmax)
    
    fig.show()

In [ ]:
plot_slideshow(np.swapaxes(img,0,1)[0],zmax=4000)

# Dot detection

In [ ]:
import datapipeline
import trackpy

In [ ]:
#pipeline function
from datapipeline.dot_detection.dot_detectors_3d import hist_jump_3d

In [ ]:
#set paths
tiff_src = '/groups/CaiLab/personal/Lex/raw/2020-08-08-takei/notebook_pyfiles/sliced_img/HybCycle_1/MMStack_Pos0.ome.tif'

In [ ]:

#Run Dot Detection - returns df of dots and tiff file 
#-----------------------------------
df_tiff_0 = hist_jump_3d.get_dots_for_tiff(tiff_src, 
                                               channels_to_detect_dots = [0+1], 
                                               
                                               #Params for Biggest Jump
                                               #----------------------------
                                               strictness = 10,
                                               nbins = 40,
                                               #----------------------------
                                               
                                               #Parameters that go into blob log 
                                               #----------------------------
                                               min_sigma = 1.5,
                                               max_sigma = 5, 
                                               num_sigma = 5,
                                               threshold = 0.001,
                                               overlap = .6,
                                               #----------------------------
                                               bool_gaussian_fitting = False,
                                           
                                               #Background Subtraction
                                               #----------------------------
                                               bool_background_subtraction = False,
                                               
                                               #You have to run an analysis with alignment 
                                               # in order to align the background to the hyb
                                               analysis_name = None,
                                               #----------------------------
                                               
                                               #Remove Super Brights, Setting to "False" does not remove any brights
                                               #----------------------------
                                               bool_remove_bright_dots = False,
                                               #----------------------------
                                               
                                               
                                               #Preprocess params
                                               #----------------------------
                                               bool_blur = False,
                                               blur_kernel_size = 3,
                                               
                                               bool_rolling_ball = False,
                                               rolling_ball_kernel_size = 20,
                                               
                                               bool_tophat = False,
                                               tophat_kernel_size = 4
                                               #----------------------------                                               
                                              )
#-----------------------------------

In [ ]:
df_tiff_0[0]

# Plot Dots

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d, img_2d, zmax):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d.x,
        y=df_locs_2d.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4,
            )
        )
    )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff, tiff, channel,raw_scr = None, raw_image = False, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel])):
            df_locs_2d = df_tiff[(df_tiff.z > z-1) & (df_tiff.z < z+1)]

            plot_2d_locs_on_2d_image(df_locs_2d, tiff[z, channel], zmax=zmax)
    else:
        
        tiff = tf.imread(raw_src)
        
        for z in range(len(tiff[:,channel])):
            df_locs_2d = df_tiff[(df_tiff.z > z-1) & (df_tiff.z < z+1)]

            plot_2d_locs_on_2d_image(df_locs_2d, tiff[z, channel], zmax=zmax)

In [ ]:
plot_3d_locs_on_2d_image(df_tiff_0[0], df_tiff_0[1], channel=0, raw_scr = None, raw_image = False, zmax=3000)

In [ ]:
raw_scr = "/groups/CaiLab/personal/Lex/raw/2020-08-08-takei/HybCycle_0/MMStack_Pos0.ome.tif"
plot_3d_locs_on_2d_image(df_tiff_0[0], df_tiff_0[1], channel=0, raw_scr = raw_src, raw_image = True, zmax=3000)

# Detect All Test

In [ ]:
import glob
import re
import numpy as np
#all paths for aligned and corrected hybs
hyb_paths = []
for i in range(12):
    directory = "/groups/CaiLab/personal/Lex/raw/20k_dash_063021_3t3/notebook_pyfiles/deconvoluted_images/HybCycle_{}/*.tif".format(i)
    paths = glob.glob(directory)
    key = [int(re.search('MMStack_Pos(\\d+)', f).group(1)) for f in paths]
    paths_fin = list(np.array(paths)[np.argsort(key)])
    hyb_paths.append(paths_fin)

In [ ]:
from dot_detection import dot_detection

In [ ]:
hyb_paths[0][0]

In [ ]:
dot_detection(hyb_paths[0][0], num_channels=4)

In [ ]:
import pandas as pd
df = pd.read_csv("./dots_detected/HybCycle_0/MMStack_Pos0.csv", index_col = 0)

In [ ]:
df

The number of dots look good.